# Shared Task: Critical thinking generation
## Loadind Data


In [24]:
import tqdm
# Reading the data
import json
import pandas as pd

def get_st_data(name : str = "validation"):
    with open(f'data_splits/{name}.json') as f:
        dataset=json.load(f)
        return dataset

for key, line in tqdm.tqdm(get_st_data("sample").items()):
    print(key)

    print(line['intervention'])
    input_text = line['intervention']
    print("-"*50)

100%|██████████| 5/5 [00:00<00:00, 42027.09it/s]

CLINTON_1_1
CLINTON: "The central question in this election is really what kind of country we want to be and what kind of future we 'll build together
Today is my granddaughter 's second birthday
I think about this a lot
we have to build an economy that works for everyone , not just those at the top
we need new jobs , good jobs , with rising incomes
I want us to invest in you
I want us to invest in your future
jobs in infrastructure , in advanced manufacturing , innovation and technology , clean , renewable energy , and small business
most of the new jobs will come from small business
We also have to make the economy fairer
That starts with raising the national minimum wage and also guarantee , finally , equal pay for women 's work
I also want to see more companies do profit-sharing"
--------------------------------------------------
Javier_84
Javier: "I have no problem requiring the airlines to give notification of any "known" delays within 30 minutes of when they become aware of it.


In [2]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [3]:

# from helper.custom_lllm import CustomLLM
import pandas as pd
from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv
import os, getpass

from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
import operator
from typing import  Annotated
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


## Prerequisites
env, LLM, ...

In [7]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("MISTRAL_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "shared_task_critical_questions"


In [12]:
model_name = "o3-mini"
# temperature = 0 not for o3-mini
temperature = "NA"

In [13]:

EXPERIMENT_NAME = f"{model_name}_temperature{temperature}"
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model=model_name, temperature=None) 

# model_api = ChatMistralAI(model="open-mistral-7b")

## Defining Workflow
####  1. Zero Shot LLM to start with...

In [ ]:


from my_models.graph_tools import build_zero_shot_graph
# memory = MemorySaver()
graph =build_zero_shot_graph(llm) #interrupt_before=['human_feedback'], checkpointer=memory)

# View
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

IndentationError: expected an indented block after function definition on line 38 (graph_tools.py, line 38)

In [19]:
import time
from contextlib import contextmanager

@contextmanager
def timer(label: str):
    start = time.perf_counter()
    try:
        yield
    finally:
        end = time.perf_counter()
        print(f"{label}: {end - start:.4f} seconds")

## Generating

In [ ]:

def _invoke_graph(params, thread= None):
    questions = []
    for event in graph.stream(params, thread, stream_mode="values"):
        # Review
        _labels = event.get('critical_question_list', '')
        if _labels:  
            questions.append(_labels)
    assert len(questions) == 1
    return questions[0]


def run( data_type: str = "validation", experiment_name: str = "", save: bool = True):
    out = {}
    for _,line in tqdm.tqdm(get_st_data(data_type).items()):
        with timer(f"Iteration {line['intervention_id']}"):
            input_arg = line['intervention']
            cqs = _invoke_graph({"input_arg": input_arg}).model_dump()['critical_questions']        
        
            for e in cqs:
                e["cq"] = e.pop("critical_question")
            line['cqs'] = cqs

            out[line['intervention_id']]=line
    if save:
        with open(f'output/output_{experiment_name}.json', 'w') as o:
            json.dump(out, o, indent=4)

    return out


# 17 min 28.4 seconds for 186 entries on o3-mini

import re

# Read the entire file content
with open('my_folder/timelogs_0shot_o3mini.txt', 'r') as file:
    data = file.read()

# Use regex to extract numbers after the colon and before "seconds"
numbers = re.findall(r'Iteration\s+\S+:\s+([\d.]+)\s+seconds', data)


df = pd.DataFrame(list(enumerate(numbers)), columns=["Iteration", f"time_{EXPERIMENT_NAME}"])
df[f"time_{EXPERIMENT_NAME}"] = pd.to_numeric(df[f"time_{EXPERIMENT_NAME}"])


,Iteration,time_o3-mini_temperatureNA
0,0,4.7111
1,1,7.0396
2,2,7.3274
3,3,4.8122
4,4,5.7671
...,...,...
181,181,5.6682
182,182,5.9174
183,183,6.4663
184,184,4.0835


## Evaluating

### Run shell 

```shell

python3 eval_scripts/evaluation.py \
    --metric similarity \
    --input_path data_splits/validation.json \
    --submission_path output/output_o3-mini_temperatureNA.json \
    --threshold 0.6 

```

#### OUTPUT : output/output_o3-mini_temperatureNA._eval_similarity_06.json



#### Overall count

| **Questions Labels** |  **#**  | **ratio** |
|:--------------------|-------:|---------:|
| useful               | **329** |  **0,59** |
| unhelpful            |      63 |      0,11 |
| Invalid              |       8 |      0,01 |
| Not able to evaluate |     158 |      0,28 |
| **Total**            | **558** |         1 |


#### Within argument 

| **n/3 useful questions per arg** | **# of arguments** | **ratio** |
|:-------------------------------:|-------------------:|:---------:|
|                               0/3 |                 17 |      0,10 |
|                               1/3 |                 51 |      0,27 |
|                               2/3 |             **76** |      0,40 |
|                               3/3 |                 42 |      0,23 |
| **Total**                       |            **186** |         1 |